# Face Recognition for the Happy House

Welcome to the first assignment of week 4! Here you will build a face recognition system. Many of the ideas presented here are from [FaceNet](https://arxiv.org/pdf/1503.03832.pdf). In lecture, we also talked about [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf).

Face recognition problems commonly fall into two categories:

- **Face Verification** - "is this the claimed person?". For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem.
- **Face Recognition** - "who is this person?". For example, the video lecture showed a face recognition video (https://www.youtube.com/watch?v=wr4rx0Spihs) of Baidu employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem.

FaceNet learns a neural network that encodes a face image into a vector of 128 numbers. By comparing two such vectors, you can then determine if two pictures are of the same person.
    
**In this assignment, you will:**
- Implement the triplet loss function
- Use a pretrained model to map face images into 128-dimensional encodings
- Use these encodings to perform face verification and face recognition

In this exercise, we will be using a pre-trained model which represents ConvNet activations using a "channels first" convention, as opposed to the "channels last" convention used in lecture and previous programming assignments. In other words, a batch of images will be of shape $(m, n_C, n_H, n_W)$ instead of $(m, n_H, n_W, n_C)$. Both of these conventions have a reasonable amount of traction among open-source implementations; there isn't a uniform standard yet within the deep learning community.

Let's load the required packages.


In [ ]:

import zipfile
with zipfile.ZipFile('weights.zip', 'r') as zip_ref:
    zip_ref.extractall('weights')



In [ ]:
#pip install fr_utils
import fr_utils

In [ ]:

from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
#from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
# from keras.layers.merge import Concatenate
from tensorflow.keras.layers import concatenate

from tensorflow.keras.layers import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
# from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer, InputSpec
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

import tensorflow
print(tensorflow.__version__)

# np.set_printoptions(threshold=np.nan)

2.17.0


## 0 - Naive Face Verification

In Face Verification, you're given two images and you have to tell if they are of the same person. The simplest way to do this is to compare the two images pixel-by-pixel. If the distance between the raw images are less than a chosen threshold, it may be the same person!

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/pixel_comparison.png" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> </u></center></caption>

Of course, this algorithm performs really poorly, since the pixel values change dramatically due to variations in lighting, orientation of the person's face, even minor changes in head position, and so on.

You'll see that rather than using the raw image, you can learn an encoding $f(img)$ so that element-wise comparisons of this encoding gives more accurate judgements as to whether two pictures are of the same person.

## 1 - Encoding face images into a 128-dimensional vector

### 1.1 - Using an ConvNet  to compute encodings

The FaceNet model takes a lot of data and a long time to train. So following common practice in applied deep learning settings, let's just load weights that someone else has already trained. The network architecture follows the Inception model from [Szegedy *et al.*](https://arxiv.org/abs/1409.4842). We have provided an inception network implementation. You can look in the file `inception_blocks.py` to see how it is implemented (do so by going to "File->Open..." at the top of the Jupyter notebook).  


The key things you need to know are:

- This network uses 96x96 dimensional RGB images as its input. Specifically, inputs a face image (or batch of $m$ face images) as a tensor of shape $(m, n_C, n_H, n_W) = (m, 3, 96, 96)$
- It outputs a matrix of shape $(m, 128)$ that encodes each input face image into a 128-dimensional vector

Run the cell below to create the model for face images.

In [ ]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model

def faceRecoModel(input_shape):
    X_input = Input(input_shape)

    # Your model's layers here...

    # Example of the last few layers
    X = Dense(128, name='dense_layer')(X_input)
    X = Lambda(lambda x: K.l2_normalize(x, axis=1), output_shape=(128,))(X)

    # Create model instance
    model = Model(inputs=X_input, outputs=X, name='FaceRecoModel')

    return model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

In [ ]:
print("Total Params:", FRmodel.count_params())

Total Params: 12416


** Expected Output **
<table>
<center>
Total Params: 3743280
</center>
</table>


By using a 128-neuron fully connected layer as its last layer, the model ensures that the output is an encoding vector of size 128. You then use the encodings the compare two face images as follows:

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/distance_kiank.png" style="width:680px;height:250px;">
<caption><center> <u> <font color='purple'>  <br> </u> <font color='purple'> By computing a distance between two encodings and thresholding, you can determine if the two pictures represent the same person</center></caption>

So, an encoding is a good one if:
- The encodings of two images of the same person are quite similar to each other
- The encodings of two images of different persons are very different

The triplet loss function formalizes this, and tries to "push" the encodings of two images of the same person (Anchor and Positive) closer together, while "pulling" the encodings of two images of different persons (Anchor, Negative) further apart.

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/triplet_comparison.png" style="width:280px;height:150px;">
<br>
<caption><center> <u> <font color='purple'> <br> </u> <font color='purple'> In the next part, we will call the pictures from left to right: Anchor (A), Positive (P), Negative (N)  </center></caption>



### 1.2 - The Triplet Loss

For an image $x$, we denote its encoding $f(x)$, where $f$ is the function computed by the neural network.

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/f_x.png" style="width:380px;height:150px;">

<!--
We will also add a normalization step at the end of our model so that $\mid \mid f(x) \mid \mid_2 = 1$ (means the vector of encoding should be of norm 1).
!-->

Training will use triplets of images $(A, P, N)$:  

- A is an "Anchor" image--a picture of a person.
- P is a "Positive" image--a picture of the same person as the Anchor image.
- N is a "Negative" image--a picture of a different person than the Anchor image.

These triplets are picked from our training dataset. We will write $(A^{(i)}, P^{(i)}, N^{(i)})$ to denote the $i$-th training example.

You'd like to make sure that an image $A^{(i)}$ of an individual is closer to the Positive $P^{(i)}$ than to the Negative image $N^{(i)}$) by at least a margin $\alpha$:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

You would thus like to minimize the following "triplet cost":

$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

Here, we are using the notation "$[z]_+$" to denote $max(z,0)$.  

Notes:
- The term (1) is the squared distance between the anchor "A" and the positive "P" for a given triplet; you want this to be small.
- The term (2) is the squared distance between the anchor "A" and the negative "N" for a given triplet, you want this to be relatively large, so it thus makes sense to have a minus sign preceding it.
- $\alpha$ is called the margin. It is a hyperparameter that you should pick manually. We will use $\alpha = 0.2$.

Most implementations also normalize the encoding vectors  to have norm equal one (i.e., $\mid \mid f(img)\mid \mid_2$=1); you won't have to worry about that here.

**Exercise**: Implement the triplet loss as defined by formula (3). Here are the 4 steps:
1. Compute the distance between the encodings of "anchor" and "positive": $\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. Compute the distance between the encodings of "anchor" and "negative": $\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. Compute the formula per training example: $ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
3. Compute the full formula by taking the max with zero and summing over the training examples:
$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2+ \alpha \large ] \small_+ \tag{3}$$

Useful functions: `tf.reduce_sum()`, `tf.square()`, `tf.subtract()`, `tf.add()`, `tf.reduce_mean`, `tf.maximum()`.

In [ ]:
# GRADED FUNCTION: triplet_loss

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)

    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)

    Returns:
    loss -- real number, value of the loss
    """

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)
    ### END CODE HERE ###

    return loss

In [ ]:
# with tf.Session() as test:
import tensorflow as tf
with tf.compat.v1.Session() as test:
#    tf.set_random_seed(1)
    tf.random.set_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random.normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random.normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)

    print("loss = " + str(loss.eval()))



loss = 350.02734


**Expected Output**:

<table>
    <tr>
        <td>
            **loss**
        </td>
        <td>
           350.026
        </td>
    </tr>

</table>

## 2 - Loading the trained model

FaceNet is trained by minimizing the triplet loss. But since training requires a lot of data and a lot of computation, we won't train it from scratch here. Instead, we load a previously trained model. Load a model using the following cell; this might take a couple of minutes to run.

Here're some examples of distances between the encodings between three individuals:

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/distance_matrix.png" style="width:380px;height:200px;">
<br>
<caption><center> <u> <font color='purple'> </u> <br>  <font color='purple'> Example of distance outputs between three individuals' encodings</center></caption>

Let's now use this model to perform face verification and face recognition!

In [ ]:
FRmodel.summary()

Model: "FaceRecoModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 3, 96, 96)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_layer (Dense)                  │ (None, 3, 96, 128)          │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_2 (Lambda)                    │ (None, 128)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,416 (48.50 KB)

 Trainable params: 12,416 (48.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
WEIGHTS = {
    'dense_layer': 'path/to/dense_layer_weights.csv',
    # No weight for Lambda layer as it’s typically used for operations, not storing weights
}

In [ ]:
def load_weights_from_FaceNet(FRmodel):
    weights = WEIGHTS  # Dictionary with weight file paths
    weights_dict = load_weights()  # Function to load weights from files

    # Set layer weights of the model
    for name in weights:
        try:
            layer = FRmodel.get_layer(name)
            if layer is not None:
                layer.set_weights(weights_dict[name])
        except ValueError as e:
            print(f"Layer name {name} not found in the model. Error: {e}")

In [ ]:
# Define the model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))

# Compile the model
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])

# Load the pre-trained weights
load_weights_from_FaceNet(FRmodel)

KeyError: 'conv1_w'

## 3 - Applying the model

Back to the Happy House! Residents are living blissfully since you implemented happiness recognition for the house in an earlier assignment.  

However, several issues keep coming up: The Happy House became so happy that every happy person in the neighborhood is coming to hang out in your living room. It is getting really crowded, which is having a negative impact on the residents of the house. All these random happy people are also eating all your food.

So, you decide to change the door entry policy, and not just let random happy people enter anymore, even if they are happy! Instead, you'd like to build a **Face verification** system so as to only let people from a specified list come in. To get admitted, each person has to swipe an ID card (identification card) to identify themselves at the door. The face recognition system then checks that they are who they claim to be.

### 3.1 - Face Verification

Let's build a database containing one encoding vector for each person allowed to enter the happy house. To generate the encoding we use `img_to_encoding(image_path, model)` which basically runs the forward propagation of the model on the specified image.

Run the following code to build the database (represented as a python dictionary). This database maps each person's name to a 128-dimensional encoding of their face.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

def faceRecoModel(input_shape):
    X_input = Input(input_shape)
    X = Dense(128, name='dense_layer')(X_input)
    X = Lambda(lambda x: K.l2_normalize(x, axis=1), output_shape=(128,))(X)
    model = Model(inputs=X_input, outputs=X, name='FaceRecoModel')
    return model

def img_to_encoding(image_path, model):
    # Dummy implementation, replace with actual image encoding function
    img = np.random.rand(96, 96, 3)  # Replace with actual image loading and preprocessing
    img = np.around(np.transpose(img, (2, 0, 1)) / 255.0, decimals=12)
    x_train = np.array([img])
    encoding = model.predict_on_batch(x_train)
    return encoding

# Initialize the model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.compile(optimizer='adam', loss='triplet_loss', metrics=['accuracy'])

# Build the database
def build_database(image_paths, model):
    database = {}
    for name, path in image_paths.items():
        try:
            encoding = img_to_encoding(path, model)
            database[name] = encoding
            print(f"Encoded {name}'s face and added to database.")
        except Exception as e:
            print(f"Error encoding {name}'s face: {e}")
    return database

# Dictionary with names and paths to images
image_paths = {
    "danielle": "images/danielle.png",
    "younes": "images/younes.jpg",
    "tian": "images/tian.jpg",
    "andrew": "images/andrew.jpg",
    "kian": "images/kian.jpg",
    "dan": "images/dan.jpg",
    "sebastiano": "images/sebastiano.jpg",
    "bertrand": "images/bertrand.jpg",
    "kevin": "images/kevin.jpg",
    "felix": "images/felix.jpg",
    "benoit": "images/benoit.jpg",
    "arnaud": "images/arnaud.jpg"
}

# Build the database
database = build_database(image_paths, FRmodel)

# Example to print the database structure
for name, encoding in database.items():
    print(f"{name}: {encoding.shape}")

Encoded danielle's face and added to database.
Encoded younes's face and added to database.
Encoded tian's face and added to database.
Encoded andrew's face and added to database.
Encoded kian's face and added to database.
Encoded dan's face and added to database.
Encoded sebastiano's face and added to database.
Encoded bertrand's face and added to database.
Encoded kevin's face and added to database.
Encoded felix's face and added to database.
Encoded benoit's face and added to database.
Encoded arnaud's face and added to database.
danielle: (1, 3, 96, 128)
younes: (1, 3, 96, 128)
tian: (1, 3, 96, 128)
andrew: (1, 3, 96, 128)
kian: (1, 3, 96, 128)
dan: (1, 3, 96, 128)
sebastiano: (1, 3, 96, 128)
bertrand: (1, 3, 96, 128)
kevin: (1, 3, 96, 128)
felix: (1, 3, 96, 128)
benoit: (1, 3, 96, 128)
arnaud: (1, 3, 96, 128)


Now, when someone shows up at your front door and swipes their ID card (thus giving you their name), you can look up their encoding in the database, and use it to check if the person standing at the front door matches the name on the ID.

**Exercise**: Implement the verify() function which checks if the front-door camera picture (`image_path`) is actually the person called "identity". You will have to go through the following steps:
1. Compute the encoding of the image from image_path
2. Compute the distance about this encoding and the encoding of the identity image stored in the database
3. Open the door if the distance is less than 0.7, else do not open.

As presented above, you should use the L2 distance (np.linalg.norm). (Note: In this implementation, compare the L2 distance, not the square of the L2 distance, to the threshold 0.7.)

In [ ]:
# GRADED FUNCTION: verify

def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".

    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be a resident of the Happy house.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras

    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """

    ### START CODE HERE ###

    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)

    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])

    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False

    ### END CODE HERE ###

    return dist, door_open

Younes is trying to enter the Happy House and the camera takes a picture of him ("images/camera_0.jpg"). Let's run your verification algorithm on this picture:

<img src="//github.com/enggen/Deep-Learning-Coursera/raw/1407e19c98833d2686a0748db26b594f3102301e/Convolutional Neural Networks/Week4/Face Recognition/images/camera_0.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's not younes, please go away


(103.26127, False)

**Expected Output**:

<table>
    <tr>
        <td>
            **It's younes, welcome home!**
        </td>
        <td>
           (0.65939283, True)
        </td>
    </tr>

</table>

Benoit, who broke the aquarium last weekend, has been banned from the house and removed from the database. He stole Kian's ID card and came back to the house to try to present himself as Kian. The front-door camera took a picture of Benoit ("images/camera_2.jpg). Let's run the verification algorithm to check if benoit can enter.
<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [ ]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(104.054214, False)

**Expected Output**:

<table>
    <tr>
        <td>
            **It's not kian, please go away**
        </td>
        <td>
           (0.86224014, False)
        </td>
    </tr>

</table>

### 3.2 - Face Recognition

Your face verification system is mostly working well. But since Kian got his ID card stolen, when he came back to the house that evening he couldn't get in!

To reduce such shenanigans, you'd like to change your face verification system to a face recognition system. This way, no one has to carry an ID card anymore. An authorized person can just walk up to the house, and the front door will unlock for them!

You'll implement a face recognition system that takes as input an image, and figures out if it is one of the authorized persons (and if so, who). Unlike the previous face verification system, we will no longer get a person's name as another input.

**Exercise**: Implement `who_is_it()`. You will have to go through the following steps:
1. Compute the target encoding of the image from image_path
2. Find the encoding from the database that has smallest distance with the target encoding.
    - Initialize the `min_dist` variable to a large enough number (100). It will help you keep track of what is the closest encoding to the input's encoding.
    - Loop over the database dictionary's names and encodings. To loop use `for (name, db_enc) in database.items()`.
        - Compute L2 distance between the target "encoding" and the current "encoding" from the database.
        - If this distance is less than the min_dist, then set min_dist to dist, and identity to name.

In [ ]:
import numpy as np
from fr_utils import img_to_encoding

def who_is_it(image_path, database, model):
    """
    Given an image_path and a database of known faces, return who the person in the image is.
    """
    # Step 1: Compute the encoding for the image
    encoding = img_to_encoding(image_path, model)

    # Initialize variables to store the minimum distance and the corresponding identity
    min_dist = float('inf')
    identity = None
    output_data = None

    # Step 2: Loop over the database to find the closest match
    for name, db_encoding in database.items():
        # Compute the distance between the input image encoding and the database image encoding
        dist = np.linalg.norm(encoding - db_encoding)

        # Check if the distance is the smallest one seen so far
        if dist < min_dist:
            min_dist = dist
            identity = name
            output_data = dist

    # Check if identity was found
    if identity is not None:
        print(f"it's {identity}, the distance is {min_dist}")
    else:
        print("Not in the database.")

    return min_dist, identity, output_data

Younes is at the front-door and the camera takes a picture of him ("images/camera_0.jpg"). Let's see if your who_it_is() algorithm identifies Younes.

In [ ]:
output = who_is_it("images/camera_0.jpg", database, FRmodel)

it's kevin, the distance is 95.46762084960938


**Expected Output**:

<table>
    <tr>
        <td>
            **it's younes, the distance is 0.659393**
        </td>
        <td>
           (0.65939283, 'younes')
        </td>
    </tr>

</table>

In [ ]:
#dictionary contains the L2 distance between target image encoding and database embeddings of other images
output[2]

95.46762

Question 6

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input

def img_to_encoding(image_path, model):
    # Load the image with target size (96, 96)
    img = image.load_img(image_path, target_size=(96, 96))

    # Convert image to NumPy array
    img = image.img_to_array(img)

    # Normalize the image
    img = np.around(img / 255.0, decimals=12)

    # Convert from HWC to CHW format
    img = np.transpose(img, (2, 0, 1))

    # Add batch dimension
    img = np.expand_dims(img, axis=0)

    # Debugging: Print shape of the input tensor
    print(f"Input shape to model: {img.shape}")

    # Perform prediction
    embedding = model.predict_on_batch(img)

    return embedding

# Load model (assuming FRmodel is defined properly)
# FRmodel = load_model('path_to_your_model.h5')

# Add new person to database
try:
    lil_nadun_encoding = img_to_encoding("images/lil_nadun.jpg", FRmodel)
    database["X"] = lil_nadun_encoding
except Exception as e:
    print(f"Error: {e}")

Input shape to model: (1, 96, 3, 96)


In [ ]:
import tensorflow as tf

def resize_image(image_path, target_size=(96, 96)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

resized_image = resize_image("images/kibs.jpg")

In [ ]:
import numpy as np
import tensorflow as tf

def resize_image(image_path, target_size=(96, 96)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def img_to_encoding(image_path, model):
    # Resize the image
    img = resize_image(image_path)
    img = np.expand_dims(img, axis=0)
    print(f"Input shape to model: {img.shape}")

    # Transpose the dimensions to (batch_size, channels, height, width)
    img = tf.transpose(img, perm=[0, 3, 1, 2])  # Transpose to (batch_size, channels, height, width)

    # Perform prediction
    embedding = model.predict_on_batch(img)
    return embedding

In [ ]:
database = {}
x_encoding = img_to_encoding("images/lil_nadun.jpg", FRmodel)
database["X"] = x_encoding

Input shape to model: (1, 96, 96, 3)


In [ ]:
# Encode the new test image
test_image_encoding = img_to_encoding("images/kibs.jpg", FRmodel)

# Perform face verification
def verify_face(test_encoding, database_encoding, threshold=0.7):
    dist = np.linalg.norm(test_encoding - database_encoding)
    return dist < threshold

is_verified = verify_face(test_image_encoding, database["X"], threshold=0.8)
print("Face Verified:", is_verified)

Input shape to model: (1, 96, 96, 3)
Face Verified: False


In [ ]:
output = who_is_it("images/kibs.jpg", database, FRmodel)
print(output)

Input shape to model: (1, 96, 96, 3)
it's X, the distance is 85.95157623291016
(85.95158, 'X', 85.95158)


In [ ]:
print("Database content:", database)

Database content: {'X': array([[[[ 0.6278404 , -0.6213507 , -0.7179638 , ...,  0.68538916,
           0.63683033,  0.6515854 ],
         [ 0.7567249 , -0.7125553 , -0.7101299 , ...,  0.9228122 ,
           0.79105127,  0.6582928 ],
         [ 0.84727466, -0.8148584 , -0.9571995 , ...,  0.83480287,
           0.73597616,  0.7490001 ],
         ...,
         [ 0.86959857, -0.81234866, -0.82805854, ...,  0.833406  ,
           0.78321457,  0.68893915],
         [ 0.8118683 , -0.83166856, -0.95832986, ...,  0.8017115 ,
           0.8178092 ,  0.30833632],
         [ 0.9321689 , -0.9849271 , -0.7726298 , ...,  0.9956527 ,
           0.9625151 , -0.3913779 ]],

        [[ 0.63446546, -0.62096566, -0.66899425, ...,  0.6790259 ,
           0.62842166,  0.63223624],
         [ 0.54231405, -0.54691124, -0.57314414, ...,  0.3807869 ,
           0.5565355 , -0.13414444],
         [ 0.44057497, -0.47997144, -0.19246806, ...,  0.45020792,
           0.5481759 ,  0.59064573],
         ...,
         [

In [ ]:
print("Database Encoding:", database["X"])
print("Test Image Encoding:", test_image_encoding)

Database Encoding: [[[[ 0.6278404  -0.6213507  -0.7179638  ...  0.68538916  0.63683033
     0.6515854 ]
   [ 0.7567249  -0.7125553  -0.7101299  ...  0.9228122   0.79105127
     0.6582928 ]
   [ 0.84727466 -0.8148584  -0.9571995  ...  0.83480287  0.73597616
     0.7490001 ]
   ...
   [ 0.86959857 -0.81234866 -0.82805854 ...  0.833406    0.78321457
     0.68893915]
   [ 0.8118683  -0.83166856 -0.95832986 ...  0.8017115   0.8178092
     0.30833632]
   [ 0.9321689  -0.9849271  -0.7726298  ...  0.9956527   0.9625151
    -0.3913779 ]]

  [[ 0.63446546 -0.62096566 -0.66899425 ...  0.6790259   0.62842166
     0.63223624]
   [ 0.54231405 -0.54691124 -0.57314414 ...  0.3807869   0.5565355
    -0.13414444]
   [ 0.44057497 -0.47997144 -0.19246806 ...  0.45020792  0.5481759
     0.59064573]
   ...
   [ 0.46476316 -0.5118498   0.39976686 ...  0.49072468  0.55574924
     0.63711184]
   [ 0.4976243  -0.4886224  -0.18690412 ...  0.50579053  0.5001849
    -0.52169687]
   [ 0.3618348  -0.02972104 -0.5136

In [ ]:
# Ensure resize_image returns the correct shape
resized_image = resize_image("images/kibs.jpg")
print(f"Resized image shape: {resized_image.shape}")

# Encode the new test image
test_image_encoding = img_to_encoding("images/kibs.jpg", FRmodel)

# Print out the encoding values
print("Test Image Encoding:", test_image_encoding)

# Perform face verification
def verify_face(test_encoding, database_encoding, threshold=0.6):
    dist = np.linalg.norm(test_encoding - database_encoding)
    print(f"Distance between test and database encodings: {dist}")
    return dist < threshold

is_verified = verify_face(test_image_encoding, database["X"])
print("Face Verified:", is_verified)

Resized image shape: (96, 96, 3)
Input shape to model: (1, 96, 96, 3)
Test Image Encoding: [[[[ 0.54160196 -0.55190164 -0.5271703  ...  0.5427011   0.5512216
     0.5416354 ]
   [ 0.5444108  -0.5513282  -0.5407419  ...  0.53423244  0.5518953
     0.5520267 ]
   [ 0.54559296 -0.5488335  -0.5430849  ...  0.53602785  0.54598516
     0.5202326 ]
   ...
   [ 0.7238326  -0.71777236  0.8624834  ...  0.79840004  0.72479874
     0.732049  ]
   [ 0.75040543 -0.6931749  -0.31445825 ...  0.88369733  0.7193371
     0.7902656 ]
   [ 0.71126723 -0.690213    0.79907286 ...  0.66234046  0.7732979
     0.8047625 ]]

  [[ 0.5809923  -0.57899535 -0.57244235 ...  0.5818887   0.5813518
     0.5874556 ]
   [ 0.57761216 -0.58163655 -0.57395047 ...  0.58204573  0.5847795
     0.6007405 ]
   [ 0.5770503  -0.57812357 -0.56596875 ...  0.57704103  0.5806222
     0.5999693 ]
   ...
   [ 0.56310594 -0.5664234   0.4990862  ...  0.5431013   0.56969804
     0.5496848 ]
   [ 0.55786204 -0.57527715 -0.55929494 ...  0.404

In [ ]:
img = resize_image("images/kibi_dogg.jpg")
print(f"Resized test image shape: {img.shape}")

Resized test image shape: (96, 96, 3)


In [ ]:
test_image_encoding = img_to_encoding("images/kibs.jpg", FRmodel)
print("Test Image Encoding Shape:", test_image_encoding.shape)

Input shape to model: (1, 96, 96, 3)
Test Image Encoding Shape: (1, 3, 96, 128)


In [ ]:
import numpy as np
import tensorflow as tf

def resize_image(image_path, target_size=(96, 96)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image

def img_to_encoding(image_path, model):
    # Resize and preprocess the image
    img = resize_image(image_path)
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Transpose to (batch_size, channels, height, width)
    img = tf.transpose(img, perm=[0, 3, 1, 2])

    # Perform prediction
    encoding = model.predict_on_batch(img)
    return encoding

def verify_face(test_encoding, database_encoding, threshold=0.7):
    dist = np.linalg.norm(test_encoding - database_encoding)
    return dist < threshold

# Test encoding
test_image_encoding = img_to_encoding("images/kibs.jpg", FRmodel)
print("Test Image Encoding Shape:", test_image_encoding.shape)

# Load database encoding (make sure this is correctly stored and retrieved)
database = {"X": np.random.rand(1, 128)}  # Example shape (1, 128), adjust as needed

# Compare with database encoding
if "X" in database:
    print("Database Encoding Shape:", database["X"].shape)
    is_verified = verify_face(test_image_encoding, database["X"])
    print("Face Verified:", is_verified)
else:
    print("No database encoding found for 'X'.")

Test Image Encoding Shape: (1, 3, 96, 128)
Database Encoding Shape: (1, 128)
Face Verified: False


You can change "`camera_0.jpg`" (picture of younes) to "`camera_1.jpg`" (picture of bertrand) and see the result.

Your Happy House is running well. It only lets in authorized persons, and people don't need to carry an ID card around anymore!

You've now seen how a state-of-the-art face recognition system works.

Although we won't implement it here, here're some ways to further improve the algorithm:
- Put more images of each person (under different lighting conditions, taken on different days, etc.) into the database. Then given a new image, compare the new face to multiple pictures of the person. This would increae accuracy.
- Crop the images to just contain the face, and less of the "border" region around the face. This preprocessing removes some of the irrelevant pixels around the face, and also makes the algorithm more robust.


<font color='blue'>
**What you should remember**:
- Face verification solves an easier 1:1 matching problem; face recognition addresses a harder 1:K matching problem.
- The triplet loss is an effective loss function for training a neural network to learn an encoding of a face image.
- The same encoding can be used for verification and recognition. Measuring distances between two images' encodings allows you to determine whether they are pictures of the same person.

Congrats on finishing this assignment!


### References:

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf)
- The pretrained model we use is inspired by Victor Sy Wang's implementation and was loaded using his code: https://github.com/iwantooxxoox/Keras-OpenFace.
- Our implementation also took a lot of inspiration from the official FaceNet github repository: https://github.com/davidsandberg/facenet
